In [ ]:
import pandas 
import numpy as np
import random

In [ ]:
def calculate_attack_damage(to_attack: int, attack_damage: str) -> int:
    damages = attack_damage.replace('d', '+').split('+')
    damage = 0
    for i in range(0,len(damages)):
        if i==1:
            damage = damage +  random.randint(1, int(damages[i]))
            #Critical hit
            if to_attack == 20:
                damage = damage +  random.randint(1, int(damages[i]))
        else:
            damage = damage + int(damages[i])
    return damage

def calculate_spell_damage(attack_damage: str) -> int:
    damages = attack_damage.replace('d', '+').split('+')
    damage = 0
    for i in range(0,len(damages)):
        if i==1:
            damage = damage +  random.randint(1, int(damages[i]))
        else:
            damage = damage + int(damages[i])
    return damage

def calculate_group_hp(creatures: list) -> int:
    creatures_hp=0
    for creature in creatures:
        creatures_hp = creatures_hp + creature.hp

    return creatures_hp

def choose_target(creatures: list):
    potential_targets = []
    for creature in creatures:
        if creature.hp>0:
            potential_targets.append(creature)
    if len(potential_targets)!=0:
        target = random.choice(potential_targets)
    else:
        target = 0

    return target

In [ ]:
# Set up
class character:
    def __init__(self, name, hp, ac, initiative_bonus, saves, heal):
        self.name = name
        self.hp = hp
        self.max_hp = hp
        self.ac = ac
        self.saves = saves
        self.initiative_bonus = initiative_bonus
        self.initiative = initiative_bonus
        self.heal = heal

    def take_attack_damage(self, attack_roll):
        to_attack, damage = attack_roll
        if to_attack >= self.ac:
            self.hp = max(self.hp - damage, 0)

    def take_heal(self, heal):
        self.hp = min(self.hp + heal, self.max_hp)
    
    def take_saving_throw_damage(self, spell_attack):
        targeted_save, spell_save_dc, damage = spell_attack
        save = random.randint(1, 20) + self.saves[targeted_save]
        if spell_save_dc > save:
            self.hp = max(self.hp - damage, 0)

    def take_damage_or_status(self, attack_info):
        attack_type, attack = attack_info
        if attack_type == "roll_to_attack":
            self.take_attack_damage(attack)
        elif attack_type == "spell_attack":
            self.take_saving_throw_damage(attack)
        elif attack_type == "take_heal":
            self.take_heal(attack)


In [ ]:
class martial(character):
  def __init__(self, name, hp, ac, attack_bonus, attack_damage, number_of_attacks, initiative_bonus, saves, heal):
    super().__init__(name, hp, ac, initiative_bonus, saves, heal)
    self.attack_bonus = attack_bonus
    self.attack_damage = attack_damage
    self.number_of_attacks = number_of_attacks

  def roll_to_attack(self):
    # Roll to attack and damage
    to_attack = random.randint(1, 20) + self.attack_bonus
    damage = calculate_attack_damage(to_attack, self.attack_damage)
    return tuple([to_attack, damage])
  
  def best_action(self):
    return tuple(["roll_to_attack", self.roll_to_attack()])

In [ ]:
class blaster(character):
  def __init__(self, name, hp, ac, spell_save_dc, attack_damage, initiative_bonus, saves, heal):
    super().__init__(name, hp, ac, initiative_bonus, saves, heal)
    self.spell_save_dc = spell_save_dc
    self.attack_damage = attack_damage
    self.targeted_save = "dex"

  def spell_attack(self):
    # Roll damage
    damage = calculate_spell_damage(self.attack_damage)
    return tuple([self.targeted_save, self.spell_save_dc, damage])

  def best_action(self):
    return tuple(["spell_attack", self.spell_attack()])


In [ ]:
class controller(character):
  def __init__(self, name, hp, ac, spell_save_dc, initiative_bonus, saves, heal):
    super().__init__(name, hp, ac, initiative_bonus, saves, heal)
    self.spell_save_dc = spell_save_dc
    self.targeted_save = "wis"

In [ ]:
hero1 = martial("barbarian", 12, 14, 2, "2d6+4", 1, 0, {"dex": 2, "wis": -1}, False)
hero2 = martial("fighter", 10, 16, 2, "1d12+4", 1, 0, {"dex": 4, "wis": 0}, False)
hero3 = blaster("cleric", 8, 12, 13, "3d6", 0, {"dex": 0, "wis": 3}, True)
# hero4 = controller("wizard", 8, 12, 14, {"dex": 1, "wis": 1}, False)
heroes = [hero1, hero2, hero3]

monster1 = martial("goblin", 12, 14, 2, "2d6+4", 1, 0, {"dex": 2, "wis": 0}, False)
monster2 = martial("goblin", 12, 14, 2, "2d6+4", 1, 0, {"dex": 2, "wis": 0}, False)
monster3 = blaster("goblin shaman", 8, 12, 13, "3d6", 0, {"dex": 1, "wis": 2}, True)
# monster4 = controller("goblin shaman", 8, 12, 14, {"dex": 1, "wis": 2}, True)
monsters = [monster1, monster2, monster3]
all_creatures = heroes + monsters

In [ ]:
initiative_dict = dict()
for creature in all_creatures:
    creature.initiative = random.randint(1, 20) + creature.initiative_bonus
    initiative_dict[creature] = creature.initiative
initiative_dict = dict(sorted(initiative_dict.items(), key=lambda item: item[1]))

In [ ]:
monsters_hp = calculate_group_hp(monsters)
heroes_hp = calculate_group_hp(heroes)
print(f"Monsters health: {monsters_hp}")
print(f"Heroes health: {heroes_hp}")
rounds=0

while (monsters_hp>0 and heroes_hp>0):
    rounds=rounds+1
    for character, _ in initiative_dict.items():
        if character.hp>0:
            if character in heroes:
                target = choose_target(monsters)
                if target == 0:
                    break
                target.take_damage_or_status(character.best_action())
            else:
                target = choose_target(heroes)
                if target == 0:
                    break
                target.take_damage_or_status(character.best_action())
    monsters_hp = calculate_group_hp(monsters)
    heroes_hp = calculate_group_hp(heroes)

if heroes_hp > monsters_hp:
    print(f"Heroes won in {rounds} rounds, they have {heroes_hp}hp remaining!")
else:
    print(f"TPK in {rounds} rounds, monsters have {monsters_hp}hp remaining!")
